In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
N = 1000000

p1 = 0.6
p2 = 0.7
A = 45
B = 6
C = 5

A_actual = (np.random.random(N) <= p1) * A
B_actual = (np.random.random(N) <= p2) * B
Result = A_actual + B_actual - C

Agent0 = (A * p1 + B * p2 - C > 0) * Result
AgentA = (A_actual + B * p2 - C > 0) * Result
AgentB = (A * p1 + B_actual - C > 0) * Result
AgentP = (A_actual + B_actual - C > 0) * Result

In [3]:
df = pd.DataFrame({
    'A': A_actual,
    'B': B_actual,
    'Result': Result,
    'Agent0': Agent0,
    'AgentA': AgentA,
    'AgentB': AgentB,
    'AgentP': AgentP,
})

In [4]:
df.mean()

A         26.998290
B          4.195926
Result    26.194216
Agent0    26.194216
AgentA    26.516554
AgentB    26.194216
AgentP    26.796196
dtype: float64

In [5]:
from scipy.stats import binom, norm, triang

In [6]:
def triang_from_bounds(minv, midv, maxv):
    scale = maxv - minv
    return triang((midv - minv) / scale, loc=minv, scale=scale)
    

In [7]:
from itertools import chain, combinations
import pandas as pd
from scipy.stats import binom, norm
import math

def powerset(iterable, max_size=5):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(min(len(s)+1, max_size+1)))

def simulate(variables, function, approx_result_size = 10000000, max_rowcount=1000000):
    combinations = list(powerset(variables))
    n = int(approx_result_size / len(combinations))
    if max_rowcount:
        n = min(max_rowcount, n)
    
    print(f"combinations = {len(combinations)}")
    print(f"rows = {n}")
    print(f"calculations = {len(combinations) * n}")

    sampled = {k: v.rvs(n) for k, v in variables.items()}
    means = {k: v.mean() for k, v in variables.items()}
    reality = function(sampled)

    agent_results = {}

    for sets in combinations:
        known_vars = set(variables) - set(sets)
        info = dict(sampled)
        info.update({k: means[k] for k in known_vars})
        decisions = function(info)
        
        if len(sets) > 0:
            agent_name = '_'.join(['known'] + list(sorted(sets)))
        else:
            agent_name = 'naive'
            
        agent_results[agent_name] = reality * (decisions > 0)
        
    return pd.DataFrame(agent_results)

#     all_data = {'reality': reality}
#     all_data.update(sampled)
#     all_data.update(agent_results)

#     return pd.DataFrame(all_data)

In [8]:

def result(v):
    return v['p1'] * 45 + v['p2'] * 8 - 5


myvariables = {
    'p1': binom(1, 0.6),
    'p2': binom(1, 0.7)
}

simulate(myvariables, result, )

combinations = 4
rows = 1000000
calculations = 4000000


,naive,known_p1,known_p2,known_p1_p2
0,40,40,40,40
1,3,3,3,3
2,3,3,3,3
3,48,48,48,48
4,-5,-5,-5,0
...,...,...,...,...
999995,48,48,48,48
999996,3,3,3,3
999997,40,40,40,40
999998,3,3,3,3


In [9]:
def result(v):
    return v['a1'] * v['p1'] + v['a2'] * v['p2'] - 500


myvariables = {
    'a1': triang_from_bounds(0, 100, 400),
    'p1': triang_from_bounds(1, 2, 3),
    'a2': triang_from_bounds(250, 300, 350),
    'p2': triang_from_bounds(-1, 1, 7),
}

df = simulate(myvariables, result)
df

combinations = 16
rows = 625000
calculations = 10000000


,naive,known_a1,known_p1,known_a2,known_p2,known_a1_p1,known_a1_a2,known_a1_p2,known_a2_p1,known_p1_p2,known_a2_p2,known_a1_a2_p1,known_a1_p1_p2,known_a1_a2_p2,known_a2_p1_p2,known_a1_a2_p1_p2
0,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351,1564.918351
1,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268,392.446268
2,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650,506.493650
3,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650,466.973650
4,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852,1171.486852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
624995,111.384981,111.384981,111.384981,111.384981,111.384981,111.384981,111.384981,111.384981,111.384981,0.000000,111.384981,111.384981,111.384981,111.384981,0.000000,111.384981
624996,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951,373.027951
624997,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081,175.889081
624998,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442,925.465442


In [10]:
(df.clip(upper=0).mean() - df.clip(upper=0).mean().naive).sort_values()

naive                 0.000000
known_a1              0.000000
known_p1              0.000000
known_a2              0.000000
known_a1_p1           0.000000
known_a1_a2           0.000000
known_a2_p1           0.000000
known_a1_a2_p1        0.000000
known_p2             28.444861
known_a2_p2          28.502275
known_p1_p2          28.891449
known_a2_p1_p2       28.963715
known_a1_p2          32.745228
known_a1_a2_p2       32.802833
known_a1_p1_p2       33.484787
known_a1_a2_p1_p2    33.541135
dtype: float64